# Import libraries

In [3]:
import numpy as np
import tensorflow as tf 
import spektral

# Load the CORA dataset

In [ ]:
dataset = spektral.datasets.citation.Citation(
    name = 'cora',
    random_split=True, 
    normalize_x = False
)

# Node features
features = dataset.read()[0].x

# Adjacency matrix
adj = dataset.read()[0].a

# Node-wise labels
labels = dataset.read()[0].y

# Train - val - test masks
train_mask = dataset.mask_tr
val_mask = dataset.mask_va
test_mask = dataset.mask_te

# Print out the shapes
print(features.shape)
print(adj.shape)
print(labels.shape)

print(np.sum(train_mask))
print(np.sum(val_mask))
print(np.sum(test_mask))

'''
Number of nodes : 2708
Number of features per node : 1433 
Number of labels (classes) : 7 

------

Number of training samples : 140
Number of validating samples : 210
Number of testing samples : 2358
'''

(2708, 1433)
